In [1]:
import requests
import calendar
import dateutil.parser as parser
import yaml
import time
import pandas as pd
import warnings
import concurrent.futures
warnings.filterwarnings('ignore')

In [2]:
def convert_date(utc_time): 
    parsed_date = parser.parse(utc_time)
    var_date=parsed_date.date()
    var_time=parsed_date.time()
    var_f_time=var_time.hour
    var_julian_date=parsed_date.timetuple().tm_yday
    var_weekday=parsed_date.weekday()
    var_weekday_name=calendar.day_name[parsed_date.weekday()]
    return var_date, var_time, var_f_time, var_julian_date, var_weekday, var_weekday_name

In [3]:
with open ('config.yml') as ymlfile:
    cfg = yaml.safe_load(ymlfile)
    oanda_api_key = cfg['creds']['oanda_api']
    account_number = cfg['creds']['account_number']

In [4]:
currency_pairs = ['EUR_USD','USD_CAD','EUR_GBP','EUR_AUD','EUR_CHF',
                  'GBP_USD','GBP_CHF','GBP_NZD','GBP_AUD','GBP_CAD',
                  'AUD_CAD','AUD_CHF','AUD_NZD','NZD_USD','EUR_CAD',
                  'USD_CHF','CAD_CHF','NZD_CAD','AUD_USD','EUR_NZD',
                  'NZD_CHF']

currency_pairs = ['EUR_USD','USD_CAD']

timeframe = "H4"
price_char = "M"
price_com = "mid"
candles_count = 5000

params_count = (
    ('price', price_char),
    ('count', candles_count),
    ('granularity', timeframe),
)

In [5]:
provider_api_url = 'https://api-fxpractice.oanda.com/v3/accounts/{}/orders'.format(account_number)
request_headers = {
    "Authorization": oanda_api_key,
    "Accept-Datetime-Format": "RFC3339",
    "Connection": "Keep-Alive",
    "Content-Type": "application/json;charset=UTF-8"
}

provider_authorization = 'Bearer {0}'.format(oanda_api_key)

headers = {
    'Content-Type': 'application/json',
    'Authorization': provider_authorization,
}

In [6]:
def get_candles(pair):
    output = []
    filename = "{}_{}.csv".format(pair, timeframe)
    first_response = requests.get('https://api-fxpractice.oanda.com/v3/instruments/{}/candles'.format(pair), 
                            headers=headers,
                            params=params_count).json()

    response=first_response['candles']    
    all_candlesticks = response

    for i in range (len(all_candlesticks)):
        result= (convert_date(response[i]['time']))
        output.append([(result[0]),(result[1]),
                    (result[2]),(result[3]),
                    (result[4]),(result[5]),
                        response[i]['time'],
                        response[i]['volume'], 
                        response[i][price_com]['o'],
                        response[i][price_com]['h'],
                        response[i][price_com]['l'],
                        response[i][price_com]['c']])
        
    output = pd.DataFrame(output)
    output.columns = ['Date','Time',
                    'f_time','julian_date',
                    'Weekday','Weekday_Name',
                    'UTC_Time', 'Volume',
                    'Open', 'High', 'Low', 'Close']
    
    output['SMA_3'] = output['Close'].rolling(window=3).mean().round(4)
    output['SMA_5'] = output['Close'].rolling(window=5).mean().round(4)
    output['SMA_6'] = output['Close'].rolling(window=6).mean().round(4)
    output['SMA_10'] = output['Close'].rolling(window=10).mean().round(4)
    output['SMA_20'] = output['Close'].rolling(window=20).mean().round(4)

    output.to_csv(filename, header = True, index = False)

In [7]:
t1 = time.perf_counter()

with concurrent.futures.ProcessPoolExecutor() as executor:
    executor.map(get_candles, currency_pairs)

t2 = time.perf_counter()

print(f'Finished in {t2-t1} seconds')

Finished in 1.2588705579996713 seconds


In [8]:
df = pd.read_csv('EUR_USD_H4.csv')

In [9]:
df.tail()

,Date,Time,f_time,julian_date,Weekday,Weekday_Name,UTC_Time,Volume,Open,High,Low,Close,SMA_3,SMA_5,SMA_6,SMA_10,SMA_20
4995,2022-02-18,02:00:00,2,49,4,Friday,2022-02-18T02:00:00.000000000Z,11727,1.13682,1.13716,1.13612,1.13659,1.1365,1.1363,1.1363,1.1366,1.1358
4996,2022-02-18,06:00:00,6,49,4,Friday,2022-02-18T06:00:00.000000000Z,22465,1.13660,1.13772,1.13640,1.13710,1.1368,1.1365,1.1364,1.1366,1.1361
4997,2022-02-18,10:00:00,10,49,4,Friday,2022-02-18T10:00:00.000000000Z,29630,1.13710,1.13725,1.13468,1.13490,1.1362,1.1363,1.1363,1.1363,1.1362
4998,2022-02-18,14:00:00,14,49,4,Friday,2022-02-18T14:00:00.000000000Z,48016,1.13491,1.13542,1.13140,1.13224,1.1347,1.1355,1.1356,1.1358,1.1361
4999,2022-02-18,18:00:00,18,49,4,Friday,2022-02-18T18:00:00.000000000Z,15439,1.13224,1.13358,1.13219,1.13237,1.1332,1.1346,1.1350,1.1354,1.1360
